# Introducción a la librería 🤗 Transformers
Este notebook es una demostración de las tareas que se pueden realizar con la librería 🤗 *transformers* de [Hugging face](https://huggingface.co)

In [ ]:
#instalamos la librería
!pip install transformers[sentencepiece]

## Uso de tareas con `pipeline`
La manera más directa de usar una tarea pre-entrenada es mediante un `pipeline`. Transformers tiene tareas pre-entrenadas para:
- Sentiment analysis: is a text positive or negative?
- Text generation (in English): provide a prompt and the model will generate what follows.
- Name entity recognition (NER): in an input sentence, label each word with the entity it represents (person, place,
  etc.)
- Question answering: provide the model with some context and a question, extract the answer from the context.
- Filling masked text: given a text with masked words (e.g., replaced by `[MASK]`), fill the blanks.
- Summarization: generate a summary of a long text.
- Translation: translate a text in another language.
- Feature extraction: return a tensor representation of the text.  

Primero importamos la clase `pipeline` antes de poder usarla:


In [ ]:
from transformers import pipeline

### Análisis de sentimientos

In [ ]:
classifier = pipeline('sentiment-analysis')

Una vez instanciado el modelo, el uso es casi inmediato:

In [ ]:
classifier('We are very happy to show you the 🤗 Transformers library.')

Podemos elegir cualquier modelo pre-entrenado del [model hub](https://huggingface.co/models) de HugginFace. Por ejemplo el modelo `"nlptown/bert-base-multilingual-uncased-sentiment"` está pre-entrenado en varios idiomas, entre ellos el español

In [ ]:
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
classifier('Me encanta el helado de vainilla')

In [ ]:
classifier('I hate chocolate ice cream')

In [ ]:
classifier(['Odio el helado de chocolate', 'Me encanta el helado de vainilla'])

### Zero-shot classification
Con esta tarea podemos clasificar un texto sin necesidad de etiquetar un conjunto de entrenamiento.

In [ ]:
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

### Generación de texto
Usando un modelo generativo (de tipo auto-regresivo) podemos generar un texto a partir de una semilla.

In [ ]:
generator = pipeline("text-generation")
generator("In this tutorial, we will teach you how to")

In [ ]:
output = generator("In this tutorial, we will teach you how to", num_return_sequences=2)
print(output[0]['generated_text'])
print(output[1]['generated_text'])

In [ ]:
generator = pipeline("text-generation", model="mrm8488/spanish-gpt2")
generator("Me llamo Joan y me gusta")

### Mask filling
Esta tarea consiste en rellenar los huecos en medio de una frase. Esta es la tarea con la que se entrenan los modelos de lenguaje de los *transformers*

In [ ]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

In [ ]:
unmasker("I went to a japanese <mask> to eat some <mask> with cheese.", top_k=1)

### Named Entity Recognition
En esta tarea se etiqueta cada *token* según su pertenencia a una entidad.

In [ ]:
ner = pipeline("ner", aggregation_strategy="simple")
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
#probar con aggregation_strategy="none" (default) para ver la etiqueta de cada token con un esquema B-I-O
ner = pipeline("ner", aggregation_strategy="none")
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

### Sistemas de respuesta automática (question answering)
Esta tarea consiste en responder una pregunta a partir de un contexto.

In [ ]:
question_answerer = pipeline("question-answering")
context = r"""
Joan lives in New York. His friend Antonio lives in Brussels.
"""
question_answerer(
    question="Where does Joan live?",
    context=context
)

In [ ]:
context[34:95]

### Generación de resúmenes (summarization)
Esta tarea consiste en generar un resumen corto (abstractivo) a partir de un texto.

In [ ]:
summarizer = pipeline("summarization")
summarizer("""
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
""")

### Traducción de texto
Se puede usar el modelo por defecto especificando el par de idiomas en el nombre de la tarea, o podemos usar un modelo específico del [model hub](https://huggingface.co/models).

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
translator("Me llamo Joan y soy profesor de universidad.")

## Uso de los modelos
Para usar estos modelos en nuestro flujo de trabajo (p. ej. como un modelo de `tensorflow.keras`) lo necesitamos cargar junto a su función de tokenizado específica.  
Por ejemplo, para un modelo de análisis de sentimientos:

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AutoConfig

nombre_modelo = "distilbert-base-uncased-finetuned-sst-2-english"
config = AutoConfig.from_pretrained(nombre_modelo)
config.output_hidden_states = True
tf_model = TFAutoModelForSequenceClassification.from_pretrained(nombre_modelo, config=config)
tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)


Para usar el modelo, primero convertimos la entrada en tokens

In [ ]:
tf_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="tf"
)

In [ ]:
#genera un diccionario con 'inputs_ids' y 'attention_mask' para cada texto de entrada
for key, value in tf_batch.items():
    print(f"{key}: {value.numpy().tolist()}")

In [ ]:

print(tokenizer.convert_ids_to_tokens(tf_batch['input_ids'][0]))

Aplicamos el modelo, que devuelve los 'logits' de la última capa y las salidas de cada capa intermedia (*embeddings*)

In [ ]:
tf_outputs = tf_model(tf_batch)
tf_outputs.keys()

In [ ]:
len(tf_outputs.hidden_states) #Nº de capas de atención internas del transformer

In [ ]:
tf_outputs.hidden_states[0].shape #embeddings de salida de cada capa (nª muestras, nº tokens, nº dimensiones)

Aplicamos la función de activación Softmax para obtener las probabilidades normalizadas de cada clase (negativo, positivo) a partir de los logits

In [ ]:
import tensorflow as tf
predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
print(predictions)

In [ ]:
import numpy as np

np.argmax(predictions, axis=1)

## Sesgo de los modelos
Los modelos de lenguaje de los *transformers* se han entrenado con grandes cantidades de texto no supervisado, mayoritariamente obtenido de Internet. Por tanto, puede tener sesgos (racismo, sesgo de género, etc.)

In [ ]:
unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])